In [3]:
import cv2
import numpy as np
from tkinter import Tk, filedialog, Label, Button
from PIL import ImageTk, Image
from roboflow import Roboflow

# Create a Roboflow object with the Roboflow API key
rf = Roboflow(api_key="8lxaxi7p5gAB8j30hNtF")

# Select the project and get the model
project = rf.workspace("test-wrdoj").project("food-nkiop")
model = project.version(1).model

# Specify class names
class_names = [
    " ","but", "domates", "ekmek", "et", "fasulye", 
    "havuc", "mantar", "mercimek", "misir", "nohut", 
    "patates", "peynir"," ", "pirinc", "salatalik", "sarimsak", 
    "sogan", "sut", "tavuk", "yumurta"
]

# Recipes
tarifler = {
    "salata": ["salatalik", "domates", "sogan","havuc","misir"],
    "mercimek çorbası": ["mercimek", "sogan", "sarimsak", "patates"],
    "kek": ["yumurta", "sut","havuc"],
    "fırın tavuk":["but","tavuk","sogan","sarimsak","patates","havuc"],
    "Kış Güvevi": ["sogan","sarimsak","patates","havuc"],
    "Nohut Köftesi":["nohut","sogan","yumurta","sarimsak"],
    "Nohut Yemeği": ["nohut","sogan","et"],
    "Kuru fasulye Yemeği": ["fasulye","sogan","havuc","sarimsak"],
    "Tavuk Pilav" : ["but","pirinc"],
    "Tavuk İç Pilav":["tavuk","pirinc","sogan","sarimsak"],
    "Patatesli Yumurta":["patates","yumurta","peynir"],
    "Salatalık Mezesi":["peynir","salatalik"],
    "Soslu Et Sote":["et","sogan","sut","sarimsak"],
    "Et Sote":["et","sogan"],
    "Mantarlı Et Sote":["mantar","et","sarimsak"],
    "Sebzeli Pilav":["pirinc","patates","havuc","sogan"],
    "Sut Mısır":["misir","sut"],
    "Çıtır But":["yumurta","but","ekmek"],
    "Sarımsaklı Peynirli Ekmek" : ["peynir","sarimsak","ekmek"],
    "Köfte":["et","ekmek","yumurta"],
    "Nohut Çorbası" : ["nohut","sogan","sarimsak"],
    "Fırında but": ["but","misir","sarimsak","sogan"],
    "Patates Salatası" : ["patates","sogan","havuc"],
    "Patatesli Börek" : ["patates","yumurta","sogan"],
    "Ekmek Köftesi" : ["ekmek","yumurta","sogan","sarimsak"],
    "Mantar Sote" : ["mantar","sogan","sarimsak"]
}

# Create the Tkinter interface
root = Tk()
root.title("Nutrient Determination and Recommended Meals")
root.geometry("800x600")

# Ask the user to select an image
def select_image():
    image_path = filedialog.askopenfilename(title="Resim Seç", filetypes=(("Image files", "*.jpg;*.jpeg;*.png"), ("All files", "*.*")))
    if image_path:
        process_image(image_path)

# Image processing and results display function
def process_image(image_path):
    # Resmi tahmin et
    prediction_results = model.predict(image_path, confidence=40, overlap=30)

    # Save the estimated official
    prediction_image_path = "prediction.jpg"
    prediction_results.save(prediction_image_path)

    # Print prediction results to the screen
    detected_objects = []
    for prediction in prediction_results:
        class_id = prediction["class_id"]
        if 0 <= class_id < len(class_names):
            class_name = class_names[class_id]
            detected_objects.append(class_name)

    # Find recommended dishes
    recommended_recipes_primary = []
    recommended_recipes_alternative = []
    for tarif, icerikler in tarifler.items():
        common_ingredients = set(detected_objects).intersection(icerikler)
        common_percentage = len(common_ingredients) / len(icerikler)
        if common_percentage >= 0.65:
            recommended_recipes_primary.append(tarif)
        elif common_percentage >= 0.50:
            recommended_recipes_alternative.append(tarif)

    # Print recommended dishes on text label
    if recommended_recipes_primary:
        recommended_primary_text = "\nMain meal recommendations:\n" + "\n".join(recommended_recipes_primary)
    else:
        recommended_primary_text = "\nMain meal recommendations:\n" + "No recipe was found."

    if recommended_recipes_alternative:
        recommended_alternative_text = "\nAlternative meal suggestions:\n" + "\n".join(recommended_recipes_alternative)
    else:
        recommended_alternative_text = "\nAlternative meal suggestions:\n" + "No recipe was found."

    # Remove iterations
    detected_objects = list(set(detected_objects))
    
    # Estimated official open
    prediction_image = cv2.imread(prediction_image_path)
    prediction_image = cv2.cvtColor(prediction_image, cv2.COLOR_BGR2RGB)
    prediction_image = Image.fromarray(prediction_image)
    prediction_image.thumbnail((400, 400))
    prediction_image = ImageTk.PhotoImage(prediction_image)
    prediction_label.config(image=prediction_image)
    prediction_label.image = prediction_image

    detected_objects_text = "\nDetected objects:\n" + "\n".join(detected_objects)
    result_label.config(text=detected_objects_text + recommended_primary_text + recommended_alternative_text)

# Create image select button
select_button = Button(root, text="Select Image", command=select_image)
select_button.pack(pady=10)

# Label to show the predicted image
prediction_label = Label(root)
prediction_label.pack(pady=10)

# Text label to show predicted objects and suggested dishes
result_label = Label(root, justify="left")
result_label.pack(pady=10)

# Start the Tkinter loop
root.mainloop()


loading Roboflow workspace...
loading Roboflow project...
